In [1]:
from qiskit_algorithms import Grover
from qiskit.quantum_info import Statevector
from qiskit_algorithms import AmplificationProblem
from qiskit import transpile, ClassicalRegister, QuantumRegister
import math
import importlib
import os
current_dir=os.getcwd()
os.chdir("../../")
import gen_utilities as utl
importlib.reload(utl)
os.chdir(current_dir)
from qiskit_aer import AerSimulator
import json
import qiskit
from qiskit_ibm_runtime import SamplerV2
from qiskit import QuantumCircuit
from qiskit.converters import (
    circuit_to_dag, 
    # dag_to_circuit
    )
from copy import deepcopy
from qiskit.circuit.library import TGate, HGate, XGate, SGate
from qiskit.synthesis import SolovayKitaevDecomposition
from qiskit.synthesis import generate_basic_approximations
from qiskit.transpiler.passes import SolovayKitaev

from qiskit.circuit import QuantumCircuit
from qiskit.synthesis import synth_qft_full

print(qiskit.__version__)

# BASIS=["cx", "rz", "h"]
# BASIS=["cx", "h", "s", "t"]

SHOTS=200000

def meas_qubits(circ: QuantumCircuit, target_qubits):
    '''Meas the desired qubits. Gives the meas label.'''
    meas_cregs=ClassicalRegister(len(target_qubits), "meas")
    circ.add_register(meas_cregs)
    for idx, q in enumerate(target_qubits):
        circ.measure(q, meas_cregs[idx])

def create_noise_est_cir(circ):
    noise_est_circ=utl.replace_gates_h_sx_rx(circ)
    return noise_est_circ

def replace_t_with_injection(circ: QuantumCircuit, num_qubits: int):
    '''Replaces t gate with injection.'''
    dag=circuit_to_dag(circ)
    throwaway_cr=ClassicalRegister(1, "throw_away")
    meas_cr=ClassicalRegister(num_qubits, "meas")
    qr=QuantumRegister(num_qubits+1)
    new_qc=QuantumCircuit(qr, meas_cr, throwaway_cr)
    op_nodes=dag.op_nodes()
    for node in op_nodes:
        # print(node.name)
        if node.name=="t" or node.name=="tdg":
            new_qc.h(qr[num_qubits])
            if node.name=="t":
                new_qc.t(qr[num_qubits])
            else:
                new_qc.tdg(qr[num_qubits])
            new_qc.cx(node.qargs[0]._index, qr[num_qubits])
            if node.name=="t":
                new_qc.cs(num_qubits, node.qargs[0]._index)
            else:
                new_qc.csdg(num_qubits, node.qargs[0]._index)
            new_qc.measure(qr[num_qubits], throwaway_cr[0])
            new_qc.reset(qr[num_qubits])

        else:
            utl.copy_node(new_qc, node)
    return new_qc

def split_circuit(circuit, start, end):
    nq = len(circuit.qubits)
    qc2 = QuantumCircuit(nq)
    for x in circuit[start:end]:
        qc2.append(x[0], x[1])
    return qc2

def ghz(num_qubits):
    circ=QuantumCircuit(num_qubits)
    circ.h(0)
    # for idx in list(range(num_qubits))[1::]:
    #     circ.cx(0, idx)
    for idx in list(range(num_qubits))[:-1:]:
        circ.cx(idx, idx+1)
    return circ

def qpe(targets, angle):
    qpe3 = QuantumCircuit(targets)

    # Apply H-Gates to counting qubits:
    for qubit in range(targets-1):
        qpe3.h(qubit)

    # Prepare our eigenstate |psi>:
    qpe3.x(targets-1)

    # Do the controlled-U operations:
    repetitions = 1
    for counting_qubit in range(targets-1):
        for i in range(repetitions):
            qpe3.cp(angle, counting_qubit, targets-1)
        repetitions *= 2

    # Do the inverse QFT:
    # print(QFT(targets-1, inverse=True))
    qpe3 = qpe3.compose(synth_qft_full(targets-1, inverse=True, do_swaps=True), range(targets-1))
    return qpe3

1.3.0


In [2]:
from qiskit_ibm_runtime import QiskitRuntimeService

# service=QiskitRuntimeService()
# backend=service.backend("ibm_torino")
# target="1"*4
# num_qubits=len(target)
# p1_error_prob=0.001
# p2_error_prob=10*p1_error_prob
# p1_depol=depolarizing_error(p1_error_prob, 1)
# p2_depol=depolarizing_error(p2_error_prob, 2)
# noise_model=NoiseModel()
# noise_model.add_all_qubit_quantum_error(p1_depol, ["measure", "x", "rz", "rx", "reset"])
# noise_model.add_all_qubit_quantum_error(p2_depol, ["cz"])

# noisy_sim=AerSimulator(method="matrix_product_state", noise_model=noise_model)
# # noisy_sim=AerSampler(options=dict(run_options=dict(noise_model=noise_model)))
# # backend=FakeTorino()
# # noisy_sim=SamplerV2(backend)

service = QiskitRuntimeService()

backend=service.backend("ibm_marrakesh")
print(backend)
noisy_sampler=SamplerV2(backend)

<IBMBackend('ibm_marrakesh')>


In [3]:
# Ideal execution
BASIS=["cz", "x", "rz", "sx"]

num_qubits=6
circ=qpe(num_qubits, 31*math.pi/16)
# oracle = Statevector.from_label(target)
# problem = AmplificationProblem(oracle, is_good_state=target)
# # grover_op=problem.grover_operator
# # optimal_num_iterations = math.floor(
# #     math.pi / (4 * math.asin(math.sqrt(len(marked_states) / 2**num_qubits)))
# # )
# optimal_num_iterations=1
# circ=Grover().construct_circuit(problem, optimal_num_iterations)
# circ=transpile(circ, basis_gates=["cx", "u"], optimization_level=3)
# basic_approx=generate_basic_approximations(basis_gates=[TGate(), HGate(), XGate(), SGate()], depth=3)
# skd=SolovayKitaev(recursion_degree=2, basic_approximations=basic_approx)
# print(circ.draw(fold=-1))
# skd=SolovayKitaev(recursion_degree=2)
# circ=skd(circ)

# circ.measure_all() #contains a barrier
meas_qubits(circ, list(range(num_qubits))) #no barrier
print(circ.draw(fold=-1))
ideal_circ=deepcopy(circ)

ideal_sim=AerSimulator()
result_ideal=ideal_sim.run(ideal_circ, shots=SHOTS).result()
ideal_counts=result_ideal.get_counts()
print(ideal_counts)
ideal_dist={k:v/SHOTS for k, v in ideal_counts.items()}


        ┌───┐                                                                                                                                                                                                                                                                                                                                                                                       ┌───┐                                                                                           ┌─┐                                          
   q_0: ┤ H ├─■───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────X─┤ H ├─■──────────────■───────────────────■────────────────────────■───────────────

In [4]:
# circ=transpile(circ, optimization_level=3, basis_gates=BASIS)
# coupling=CouplingMap(list(zip(list(range(num_qubits)), list(range(1, num_qubits+1)))))
# circ=transpile(circ, coupling_map=coupling, initial_layout=list(range(num_qubits)), seed_transpiler=0, optimization_level=0)
# print(circ.draw(fold=-1))
# circ=replace_t_with_injection(circ, num_qubits)
print(circ.draw(fold=-1))
circ=transpile(circ, backend, initial_layout=list(range(num_qubits)), seed_transpiler=0, optimization_level=3)
noise_est_circ=create_noise_est_cir(circ)
clifford_noise_est_cir=utl.circ_replace_rz(noise_est_circ, num_qubits)
clifford_noise_est_cir=utl.remove_idle_wires(clifford_noise_est_cir)
print(f"noise payload circ {circ.draw(fold=-1)}")
print(f"noise estimation circ {noise_est_circ.draw(fold=-1)}")
print("ops payload: ", circ.count_ops())
print("ops noise est cir: ", noise_est_circ.count_ops())
print("ops clifford noise est cir: ", clifford_noise_est_cir.count_ops())
      

        ┌───┐                                                                                                                                                                                                                                                                                                                                                                                       ┌───┐                                                                                           ┌─┐                                          
   q_0: ┤ H ├─■───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────X─┤ H ├─■──────────────■───────────────────■────────────────────────■───────────────

In [5]:
noisy_sampler.options.twirling.enable_measure=True
# noisy_sampler.options.dynamical_decoupling.enable=False
noisy_sampler.options.twirling.shots_per_randomization= "auto"
# noisy_sampler.options.twirling.num_randomizations= 4**num_qubits
noisy_sampler.options.twirling.strategy= "active-circuit"
noisy_sampler.options.twirling.enable_gates=True

result_cliff_noise_est=ideal_sim.run(clifford_noise_est_cir, shots=SHOTS).result()
# result_noisy=noisy_sampler.run([circ, noise_est_circ], shots=SHOTS).result()
job = service.job('cxm71r76t010008d5hn0')
result_noisy = job.result()


In [6]:
def strip_classical_reg(remove_pos: int, dist: dict):
    '''Modifies the keys of dist by splitting it the key by space and removing the strings at index remove_pos.
    This is from stripping unwanted classical registers.'''
    new_dict={}
    for k, v in dist.items():
        k=k.split(" ")
        k.pop(remove_pos)
        k="".join(k)
        count=new_dict.get(k, 0)
        count+=v
        new_dict[k]=count
    return new_dict

In [7]:
# print(result_noisy)
# counts_ideal_noise_est=strip_classical_reg(0, result_cliff_noise_est.get_counts())
# noisy_counts=strip_classical_reg(0, result_noisy.get_counts()[0])
# counts_noisy_noise_est=strip_classical_reg(0, result_noisy.get_counts()[1])

counts_ideal_noise_est=result_cliff_noise_est.get_counts()
noisy_counts=result_noisy[0].data.meas.get_counts()
counts_noisy_noise_est=result_noisy[1].data.meas.get_counts()

print(noisy_counts)
print(counts_noisy_noise_est)
dist_noisy_noise_est={k:v/SHOTS for k,v in counts_noisy_noise_est.items()}
print(counts_ideal_noise_est)

{'111111': 115834, '100000': 2885, '011111': 5016, '101111': 6039, '111101': 13842, '100001': 3344, '110111': 3678, '110011': 1044, '011110': 1207, '000000': 4263, '110000': 390, '101101': 1327, '110101': 957, '111011': 3413, '100111': 3245, '111110': 6309, '011010': 325, '110110': 640, '101001': 586, '111100': 1277, '010100': 255, '000011': 491, '100100': 458, '101011': 1397, '011101': 1136, '111001': 958, '000110': 286, '010010': 278, '000001': 885, '010110': 209, '111000': 410, '001100': 221, '110001': 596, '100110': 713, '100011': 2750, '101110': 796, '001101': 245, '010000': 575, '000010': 1093, '100101': 1254, '000101': 288, '011000': 430, '100010': 1334, '000100': 363, '101000': 355, '111010': 697, '010101': 257, '011011': 351, '010111': 286, '001011': 202, '101010': 467, '110010': 495, '001010': 265, '011001': 309, '101100': 332, '001000': 395, '010001': 203, '011100': 609, '001110': 247, '110100': 328, '001111': 427, '000111': 322, '001001': 218, '010011': 193}
{'001100': 9409

In [8]:
with open("ideal_counts_hardware.json", "w") as f:
    json.dump(ideal_counts, f)

with open("noisy_noise_est_dist_hardware.json", "w") as f:
    json.dump(dist_noisy_noise_est, f)

with open("ideal_noise_est_counts_hardware.json", "w") as f:
    json.dump(counts_ideal_noise_est, f)

with open("noisy_counts_hardware.json", "w") as f:
    json.dump(noisy_counts, f)